In [2]:
import pandas as pd
import evaluate
from bert_score import score as bertscore
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [3]:
deepseek_r1_df = pd.read_csv("deepseek_sample_summaries.csv")
reference_df = pd.read_csv("deepseek_bhc_matched_reference_summaries.csv")

df = pd.merge(deepseek_r1_df, reference_df, on="note_id")
generated = df['summary'].fillna("").tolist()
reference = df['target'].fillna("").tolist()

In [5]:
### 1. ROUGE Score
rouge = evaluate.load("rouge")
rouge_result = rouge.compute(predictions=generated, references=reference)

print("\n📊 ROUGE Scores:")
for k, v in rouge_result.items():
    print(f"{k.upper()}: {v:.4f}")


📊 ROUGE Scores:
ROUGE1: 0.3221
ROUGE2: 0.0740
ROUGEL: 0.1417
ROUGELSUM: 0.2082


In [6]:
### 2. BLEU (average over all rows)
smoothie = SmoothingFunction().method4
bleu_scores = [
    sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)
    for ref, pred in zip(reference, generated)
]
bleu_avg = sum(bleu_scores) / len(bleu_scores)

print("\n📊 BLEU Score (average over 1000 rows):")
print(f"BLEU: {bleu_avg:.4f}")


📊 BLEU Score (average over 1000 rows):
BLEU: 0.0235


In [7]:
### 3. BERTScore
P, R, F1 = bertscore(generated, reference, lang="en", verbose=True)
bert_avg = {
    "precision": P.mean().item(),
    "recall": R.mean().item(),
    "f1": F1.mean().item()
}

print("\n📊 BERTScore:")
print(f"Precision: {bert_avg['precision']:.4f}")
print(f"Recall:    {bert_avg['recall']:.4f}")
print(f"F1 Score:  {bert_avg['f1']:.4f}")

C:\Users\gp0382\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gp0382\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP 

calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████| 32/32 [00:37<00:00,  1.18s/it]


computing greedy matching.


100%|████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 18.95it/s]

done in 38.58 seconds, 25.92 sentences/sec

📊 BERTScore:
Precision: 0.8152
Recall:    0.8194
F1 Score:  0.8172
